# Conversion of TXT to MAT for gm/ID result files for GF180MCU

**(c) 2024-2025 Boris Murmann and Harald Pretl**

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0

In [1]:
import pandas as pd
import numpy as np
from numpy._core.records import fromarrays
from scipy.io import savemat

In [2]:
choice = 1  #start from 0
devices = ['gf180mcu_nmos_3v3', 'gf180mcu_pmos_3v3']

# widths used for characterization and fringe cap parameters (fringe caps are not included in ngspice output)
w = np.array([5, 5])
nfing = np.array([1, 1])

In [3]:
df_raw = pd.read_csv('techsweep_'+devices[choice]+'.txt', sep=r'\s+')
par_names = df_raw.columns.to_list()
par_prefix = par_names[1].split('[')[0]

In [4]:
# remove extra headers in file body and unwanted columns
df = df_raw.drop(['frequency', 'frequency.1'], axis=1)
df = df.apply(pd.to_numeric)

# rename columns for readability
df.columns = df.columns.str.removeprefix(par_prefix+'[')
df.columns = df.columns.str.replace(par_prefix[1:], '')
df.columns = df.columns.str.removesuffix(']')

# round sweep vectors to easily addressable values
df['l'] = df['l'].apply(lambda x: round(x*1e6, 2))
df['vgs'] = df['vgs'].apply(lambda x: round(x, 2))
df['vds'] = df['vds'].apply(lambda x: round(x, 2))
df['vbs'] = df['vbs'].apply(lambda x: round(x, 2))
df

,cdd,cgb,cgd,cgdo,cgg,cgs,cgso,css,gds,gm,gmbs,id,l,vbs,vds,vdsat,vgs,vth
0,1.275000e-17,-2.533000e-15,-1.146000e-17,7.949000e-16,2.543000e-15,1.138000e-18,7.949000e-16,-1.741000e-19,2.792000e-13,2.484000e-55,1.113000e-55,9.229000e-57,0.28,0.0,0.00,0.04673,0.0,0.7416
1,2.176000e-18,-2.162000e-15,-2.014000e-18,7.949000e-16,2.164000e-15,-5.867000e-19,7.949000e-16,-4.804000e-20,3.827000e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.28,-0.4,0.00,0.04818,0.0,0.8592
2,5.068000e-19,-1.916000e-15,-4.792000e-19,7.949000e-16,1.917000e-15,-5.120000e-19,7.949000e-16,-1.420000e-20,1.140000e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.28,-0.8,0.00,0.04928,0.0,0.9541
3,1.466000e-19,-1.738000e-15,-1.411000e-19,7.949000e-16,1.739000e-15,-3.676000e-19,7.949000e-16,-4.670000e-21,5.741000e-18,2.183000e-46,5.021000e-47,0.000000e+00,0.28,-1.2,-0.00,0.05020,0.0,1.0340
4,5.848000e-19,-2.533000e-15,-4.607000e-19,7.867000e-16,2.541000e-15,-7.731000e-18,7.949000e-16,4.911000e-18,4.566000e-14,1.922000e-13,8.549000e-14,7.141000e-15,0.28,-0.0,0.05,0.04673,-0.0,0.7410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107731,2.345000e-17,-1.138000e-14,-2.059000e-17,8.017000e-16,1.544000e-13,-1.430000e-13,8.200000e-16,1.071000e-13,2.384000e-08,1.986000e-05,7.146000e-06,2.290000e-05,10.00,-1.2,3.25,1.67600,3.3,1.2390
107732,3.017000e-17,2.196000e-15,-2.253000e-17,7.949000e-16,1.623000e-13,-1.645000e-13,8.200000e-16,1.239000e-13,3.867000e-08,2.432000e-05,1.194000e-05,3.404000e-05,10.00,-0.0,3.30,1.88600,3.3,0.7677
107733,2.769000e-17,-5.911000e-15,-2.229000e-17,7.949000e-16,1.589000e-13,-1.529000e-13,8.200000e-16,1.157000e-13,3.242000e-08,2.263000e-05,9.985000e-06,2.966000e-05,10.00,-0.4,3.30,1.82200,3.3,0.9508
107734,2.467000e-17,-9.514000e-15,-2.088000e-17,7.949000e-16,1.563000e-13,-1.468000e-13,8.200000e-16,1.106000e-13,2.733000e-08,2.117000e-05,8.392000e-06,2.600000e-05,10.00,-0.8,3.30,1.74900,3.3,1.1040


In [5]:
# sweep variable vectors
l = np.unique(abs(df['l']))
vgs = np.unique(abs(df['vgs']))
vds = np.unique(abs(df['vds']))
vbs = np.unique(abs(df['vbs']))

In [6]:
# data
# ngspice sweep order is l, vgs, vds, vbs
dims = [len(l), len(vgs), len(vds), len(vbs)]
print(dims)
id = np.reshape(df['id'].values, dims, order='C')
vt = np.reshape(df['vth'].values, dims, order='C')
gm = np.reshape(df['gm'].values, dims, order='C')
gmbs = np.reshape(df['gmbs'].values, dims, order='C')
gds = np.reshape(df['gds'].values, dims, order='C')
cgg = np.reshape(df['cgg'].values, dims, order='C') \
      + np.reshape(df['cgdo'].values, dims, order='C') \
      + np.reshape(df['cgso'].values, dims, order='C')
cgb = -np.reshape(df['cgb'].values, dims, order='C')
cgd = -np.reshape(df['cgd'].values, dims, order='C') \
      + np.reshape(df['cgdo'].values, dims, order='C')
cgs = -np.reshape(df['cgs'].values, dims, order='C') \
      + np.reshape(df['cgso'].values, dims, order='C')
cdd = np.reshape(df['cdd'].values, dims, order='C') \
      + np.reshape(df['cgdo'].values, dims, order='C')
css = np.reshape(df['css'].values, dims, order='C') \
      + np.reshape(df['cgso'].values, dims, order='C')
#sth = np.reshape(df['sid'].values, dims, order='C')
#sfl = np.reshape(df['sfl'].values, dims, order='C')

[6, 67, 67, 4]


In [7]:
dic = {
  "INFO": "GLOFO GF180MCU, 180nm CMOS, BSIM4",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vbs,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmbs,
  "GDS": gds,
  "CGG": cgg,
  "CGB": cgb,
  "CGD": cgd,
  "CGS": cgs,
  "CDD": cdd,
  "CSS": css,
  "STH": 0.0,
  "SFL": 0.0
}

dict

savemat(devices[choice]+'.mat', {devices[choice]: dic})